In [1]:
import gym
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete

In [2]:
import numpy as np
import random
import os

In [3]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# 2. Testing spaces

In [4]:
Box(0,1, shape=(3,3)).sample()

array([[0.71095836, 0.54975337, 0.5784741 ],
       [0.9772915 , 0.11324935, 0.62189233],
       [0.7011611 , 0.8008618 , 0.3080099 ]], dtype=float32)

# 3. Building an Environment

In [23]:


class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start temp
        self.state = 38 + random.randint(-3,3)
        # Set shower length
        self.shower_length = 60
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        self.state += action -1 
        # Reduce shower length by 1 second
        self.shower_length -= 1 
        
        # Calculate reward
        if self.state >=37 and self.state <=39: 
            reward =1 
        else: 
            reward = -1 
        
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = np.array([38 + random.randint(-3,3)]).astype(float)
        # Reset shower time
        self.shower_length = 60 
        return self.state



In [24]:
env = ShowerEnv()

In [25]:
env.observation_space.sample()

array([72.42013], dtype=float32)

# 4. Test Environment

In [27]:
episodes = 5
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        score += reward
#         print(reward)
    print(f'Episode: {episode} Score: {score}')
env.close()

Episode: 1 Score: -38
Episode: 2 Score: -42
Episode: 3 Score: 38
Episode: 4 Score: -38
Episode: 5 Score: -6


# 5.  Train Model

In [28]:
log_path = os.path.join('Training','Logs')
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [29]:
#training the model
model.learn(total_timesteps=100000)

Logging to Training\Logs\PPO_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -25.4    |
| time/              |          |
|    fps             | 49       |
|    iterations      | 1        |
|    time_elapsed    | 41       |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60           |
|    ep_rew_mean          | -29.3        |
| time/                   |              |
|    fps                  | 87           |
|    iterations           | 2            |
|    time_elapsed         | 46           |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0082060825 |
|    clip_fraction        | 0.0132       |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.09        |
|    explained_variance   |

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60           |
|    ep_rew_mean          | -23          |
| time/                   |              |
|    fps                  | 227          |
|    iterations           | 11           |
|    time_elapsed         | 99           |
|    total_timesteps      | 22528        |
| train/                  |              |
|    approx_kl            | 0.0075525837 |
|    clip_fraction        | 0.0866       |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.01        |
|    explained_variance   | -0.000153    |
|    learning_rate        | 0.0003       |
|    loss                 | 26.5         |
|    n_updates            | 100          |
|    policy_gradient_loss | -0.00644     |
|    value_loss           | 64.8         |
------------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_m

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60           |
|    ep_rew_mean          | -13.9        |
| time/                   |              |
|    fps                  | 292          |
|    iterations           | 21           |
|    time_elapsed         | 147          |
|    total_timesteps      | 43008        |
| train/                  |              |
|    approx_kl            | 0.0052470416 |
|    clip_fraction        | 0.0707       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.83        |
|    explained_variance   | -0.00151     |
|    learning_rate        | 0.0003       |
|    loss                 | 50.7         |
|    n_updates            | 200          |
|    policy_gradient_loss | -0.00286     |
|    value_loss           | 87.9         |
------------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | 0.72        |
| time/                   |             |
|    fps                  | 319         |
|    iterations           | 31          |
|    time_elapsed         | 198         |
|    total_timesteps      | 63488       |
| train/                  |             |
|    approx_kl            | 0.009919105 |
|    clip_fraction        | 0.154       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.878      |
|    explained_variance   | -0.101      |
|    learning_rate        | 0.0003      |
|    loss                 | 29.5        |
|    n_updates            | 300         |
|    policy_gradient_loss | -0.00952    |
|    value_loss           | 59.9        |
-----------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60  

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | 29.1        |
| time/                   |             |
|    fps                  | 333         |
|    iterations           | 41          |
|    time_elapsed         | 252         |
|    total_timesteps      | 83968       |
| train/                  |             |
|    approx_kl            | 0.015181535 |
|    clip_fraction        | 0.127       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.787      |
|    explained_variance   | -0.000846   |
|    learning_rate        | 0.0003      |
|    loss                 | 22.8        |
|    n_updates            | 400         |
|    policy_gradient_loss | 0.00103     |
|    value_loss           | 55.6        |
-----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 60      

# 6. Save Model

In [30]:
shower_path = os.path.join('Training', 'Saved Models','Shower_model_PPO')

In [31]:
model.save(shower_path)

D:\Program_Installation\anaconda\lib\site-packages\stable_baselines3\common\save_util.py:276: UserWarning: Path 'Training\Saved Models' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


# 7. Reload the saved model

In [32]:
del model

In [ ]:
shower_path = os.path.join('Training', 'Saved Models','Shower_100k_model_PPO')

In [ ]:
model = PPO.load(shower_path,env)

# 8. Testing the trainned model

In [39]:
evaluate_policy(model,env, n_eval_episodes=10)

(12.0, 58.787753826796276)